In [15]:
import torch
import numpy
from plotly import graph_objects as go

## 2.1 标量求导

In [2]:
x = torch.tensor(0., requires_grad=True)
a = 1
b = -2
c = 1
y = a*torch.pow(x,2) + b*x+c

y.backward()
dy_dx = x.grad
dy_dx

tensor(-2.)

## 2.2 非标量求导

In [3]:
# 方法 1
x = torch.tensor([[0., 0.],[1., 2.]], requires_grad=True)
a = 1
b = -2
c = 1
y = a*torch.pow(x,2) + b*x+c

gradident = torch.ones(x.size())
y.backward(gradient=gradident)
x_grad1 = x.grad

print(x_grad1)

tensor([[-2., -2.],
        [ 0.,  2.]])


In [4]:
# 方法 2

x = torch.tensor([[0., 0.],[1., 2.]], requires_grad=True)
a = 1
b = -2
c = 1
y = a*torch.pow(x,2) + b*x+c

gradident = torch.ones(x.size())
z = torch.sum(y*gradident)
z.backward()
x_grad2 = x.grad

print(x_grad2)

tensor([[-2., -2.],
        [ 0.,  2.]])


## 2.3 autograd

In [5]:
x = torch.tensor(0., requires_grad=True)
a = 1
b = -2
c = 1
y = a*torch.pow(x,2) + b*x+c

dy_dx = torch.autograd.grad(y,x,create_graph=True)[0]
print('一阶求导：',dy_dx.data)

dy2_dx2 = torch.autograd.grad(dy_dx,x,create_graph=True)[0]
print('二阶求导：',dy2_dx2.data)

一阶求导： tensor(-2.)
二阶求导： tensor(2.)


In [6]:
x1 = torch.tensor(1., requires_grad=True)
x2 = torch.tensor(2., requires_grad=True)

y1 = x1 * x2
y2 = x1 + x2

(dy1_dx1, dy1_dx2) = torch.autograd.grad(outputs=y1, inputs=[x1,x2],retain_graph=True)
print('多自变量求导 dy1_dx1：',dy1_dx1)

(dy2_dx1, dy2_dx2) = torch.autograd.grad(outputs=y2, inputs=[x1,x2],retain_graph=True)
print('多自变量求导 dy2_dx1：',dy2_dx1)

(dy12_dx1, dy12_dx2) = torch.autograd.grad(outputs=[y1,y2], inputs=[x1,x2],retain_graph=True)
print('多因变量求导，输出结果为导数和 dy12_dx1：', dy12_dx1)

多自变量求导 dy1_dx1： tensor(2.)
多自变量求导 dy2_dx1： tensor(1.)
多因变量求导，输出结果为导数和 dy12_dx1： tensor(3.)


利用SGD求最优值

In [13]:
x = torch.tensor(0., requires_grad=True)
a = 1
b = -2
c = 1
def f(x):
    return a*torch.pow(x,2) + b*x+c
optimizer = torch.optim.SGD(params=[x],lr=0.01)

for epoch in range(500):
    optimizer.zero_grad()
    y = f(x)
    y.backward()
    optimizer.step()

print('最优值：',f(x).data, x.data)

最优值： tensor(0.) tensor(1.0000)


## 2.3 动态计算图

In [26]:
w = torch.tensor([[3., 1.]], requires_grad=True)
b = torch.tensor([[3. ]], requires_grad=True)
X = torch.randn(10, 2)
Y = torch.randn(10, 1)

Y_hat = X@w.T + b
loss = torch.mean(torch.pow(Y-Y_hat, 2))
# 计算图在反向传播后被销毁
loss.backward()

# 需要保存计算图
# loss.backward(retain_graph=True)
print('loss.data',loss.data)
print('y_hat.data',Y_hat.data)

loss.data tensor(13.4190)
y_hat.data tensor([[-1.5928],
        [ 0.7944],
        [ 2.7251],
        [ 5.8685],
        [-1.7179],
        [ 3.0768],
        [ 5.2412],
        [ 2.9496],
        [ 0.6541],
        [ 5.1317]])
